<a href="https://colab.research.google.com/github/brainyvoyage/eip-notes/blob/master/Chapter_5_EIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python bit operators and function

1.  `x << y`: Returns x bits shifted left by y places. Equivalent to mulitplying x by `2**y`
2. `x >> y`: Returns x bits shifted right by x places. Equivalent to `x // 2**y`
3. `x & y` : Bitwise and
4. `x | y` : Bitwise or
5. `x ^ y` : Exclusive or. Each bit of output is same as `x` if corresponding bit in `y` is 0

## Hex sting

In [0]:
f"0x{int('11111111', 2):0x}"

'0xff'


## Highest bit set: 
The  number of the highest bit set is the highest power of 2 less than or equal to the input integer


In [24]:
import math
x = 15
highest_bit = math.floor(math.log(x, 2))
print(f"Highest bit {x}({bin(x)}) set = {highest_bit}")

Highest bit 15(0b1111) set = 3


## Count bits

## Method 1
> $x \& 1 = 1$ iff lowest bit in x is set, 0 otherwise

In [25]:
def bit_len(x):
  count = 0
  length = 0
  while x:
    count += (x & 1)  # x & 1 = 1 iff lowest bit in x is set, 0 otherwise
    length +=1
    x >>= 1
  return (count, length)

x = 6
count, length = bit_len(x)
print(f"{x}({bin(x)}) has {count} bit set and has {length} length")

6(0b110) has 2 bit set and has 3 length


### Method 2

$x \&  (x - 1)$ drops the lowest set bit of x
This method is faster than method above

In [4]:
def bit_count(x):
  count = 0
  while x:
    x &= (x - 1)  # Drops the lowest set bit of x
    count += 1
  return count

x = 56
print(f"X = {x}({bin(x)})\nX - 1 = ({bin(x - 1)})\nbit count = {bit_count(x)}")
# bit_count(56)

X = 56(0b111000)
X - 1 = (0b110111)
bit count = 3


### LowerSet: 
To determine the bit number of the lowest bit set in an integer, in twos-complement notation i & -i zeroes all but the lowest set bit. The bitLen() proceedure then determines its position. Obviously, negative numbers return the same result as their opposite. In this version, an input of 0 returns -1, in effect an error condition.


In [0]:
def lower_set(x):
  low = x & -x
  position = 0
  while low:
    low >>= 1
    position += 1
  return position

lower_set(56)

4



### testBit: 
returns a nonzero result, 2**offset, if the bit at 'offset' is one.



In [0]:
def test_bit(x: int, offset: int):
  mask = 1 << offset
  return x & mask

print(test_bit(56, 3))
print(bin(56))

8
0b111000


### setBit: 
Returns an integer with the bit at 'offset' set to 1.

In [0]:
def set_bit(x: int, offset):
  mask = 1 << offset
  return x | mask


### clearBit: 
returns an integer with the bit at 'offset' cleared.



In [0]:
def clear_bit(x: int, offset: int):
  mask = ~(1 << offset)
  return x & mask



### toggleBit: 
Returns an integer with the bit at 'offset' inverted, 0 -> 1 and 1 -> 0.


In [0]:
def toggle_bit(x: int, offset: int):
  mask = 1 << offset
  return x ^ mask


# Problems
## Compute parity
Parity = 1, if the nummber of 1s in the sequence is odd; otherwise 0

In [0]:
def parity_of(x):
  parity = 0
  while x:
    x &= (x - 1)  # Drops the lowest set bit of x
    parity = ~parity
  return -parity

## Swap bits

In [20]:
# Swap is necessary only if the bits are different

def swap_bits(num: int, bit_i: int, bit_j: int):
  if (num >> bit_i & 1) != (num >> bit_j & 1):
    bit_mask = (1 << bit_i) | (1 << bit_j)
    return num ^ bit_mask


x, i, j = 56, 1, 4
swapped = swap_bits(x, i, j)
print(f"Original = {bin(x)}({x})\nswapping {i} & {j} bits")
print(f"{bin(swapped)} ({swapped})")

Original = 0b111000(56)
swapping 1 & 4 bits
0b101010 (42)


## Closest integer with same bit count
Suppose we flip the bit at index $k_1$ and flip the bit at index $k_2, k_l > k_2$. Then the absolute value of the difference between the original integer and the new one is $2^{k_1} - 2^{k_2}$. To minimize this, 
1. we should make $k_1$ as small as possible and
2. $k_2$ as close to $k_1$.


Since we must preserve the weight, the bit at index kl has to be different from the bit in k2, otherwise the flips lead to an integer with different weight. This means the smallest kl can be is the rightmost bit that's different from the LSB, and k2 must be the very next bit. 

> **In summary, the correct approach is to swap the two rightmost consecutive bits that differ.**

In [24]:
WORD_SIZE = 64

def closest_num_with_same_bit(num: int):
  for i in range(WORD_SIZE - 1):
    if (num >> i & 1) != (num >> (i + 1) & 1):
      num ^= (1 << i) | (1 << (i + 1))
      return num
x = 56
y = closest_num_with_same_bit(x)
print(f"x = {x} ({bin(x)})\ny = {y} ({bin(y)})")

x = 56 (0b111000)
y = 52 (0b110100)


## Compute $x + y$

In [6]:
def bit_sum(num1: int, num2: int):
  temp1, temp2, _sum, cin = num1, num2, 0, 0
  k = 1 # To extract kth bit
  while temp1 or temp2:
    temp1k = num1 & k
    temp2k = num2 & k
    cout = (temp1k & temp2k) | (temp1k & cin) | (temp2k & cin)
    _sum |= (temp1k ^ temp2k ^ cin)
    k <<= 1
    cin = cout << 1
    temp1 >>=1
    temp2 >>=1
    #print(f"temp1 = {temp1}, temp2 = {temp2}, temp1k = {temp1k}," +
    #  f"temp2k = {temp2k}, cout = {cout}, k = {k}, _sum = {_sum} cin = {cin}")
  return _sum | cin

bit_sum(56,7)

63

## Compute $x \times y$

In [7]:
def bit_mult(x: int, y: int):
  _sum = 0
  while x:
    if x & 1:
      _sum = bit_sum(_sum, y)
    x >>= 1
    y <<= 1
  return _sum

bit_mult(8, 7)

56


## Compute $x/y$
Given two positive integers `x` and `y` compute $x/y$ if the only operators yon can use are addition, substraction, and shifting

1. We can compute the largest k such that $2^ky < x$, 
2. subtract $2^ky$ from x, and 
3. add $2^k$ to the quotient. 

For example, if $x = (1011)_2$ and $y = (10)_2$, then k = 2, since 2 * 22 < 11 and 2 * 23 > 11. We subtract $(1000)_2$ from $(1011)_2$ to get $(11)_2$, add $2^k = 22 = (100)_2$ to the quotient, and continue by updating x to $(11)_2$



In [12]:
def div(x: int, y: int):
  quotient = 0
  k = 32
  two_powk_y = y << k
  while x >= y:
    while two_powk_y > x:   # find largest k 
      two_powk_y >>= 1
      k -= 1
      
    quotient += 1 << k      #  adding 2^k * y to quotient
    x -= two_powk_y         #  Substracting 2^k from x
  return quotient

div(56, 7)

8

## Reverse digits


In [22]:
def reverse_digits(num: int):
  result = 0
  x = abs(num)
  while x:
    result = result * 10 + x % 10
    x //= 10
  return result if num > 0 else -result

reverse_digits(-312)


-213

## Is Palindrome

In [0]:
import math
def is_palindrome(num: int):
  num_digits = math.floor(math.log(num, 10)) + 1
  msd_mask = math.pow(10, num_digits - 1)
  
  for _ in range(0, num_digits//2):
    if num//msd_mask != num % 10:
      return False
    num %= msd_mask
    num //= 10
    msd_mask /= 100
  return True

assert is_palindrome(31213) == True
assert is_palindrome(34213) == False
assert is_palindrome(312213) == True

## Convert Base
Write a function that performs base conversion. Specifically, the input are
1. an integer base $b_1$, 
2. a string $s$, representing an integer x in base $b_1$, and 
3. another integer base $b_2$

Assume $2\leq b_1, b_2 \leq 16$



## Generate uniform random number
Implement a random number generator that generates a random integer $i$ in $[a, b]$, given a random number generator that produces either 0 or 1 with equal probability.



## The open Doors problem
Five hundred closed doors along a corridor are numbered from 1 to 500. A person walks through the corridor and opens each door. Another person walks through the corridor and closes every alternate door. Continuing in this manner, the i-th person comes and toggles the position of every i-th door starting from door i. Which doors are open after the 500-th person has walked through?




## Compute the greatest common divisior 
Design an efficient algorithm for computing the GCD of two numebrs without using multiplication, division, or the modulus operator
